# 2020 Kaggle Machine Learning & Data Science Survey
- 서베이 데이터를 이용해 하나의 스토리를 만들어내는 컴피티션이지만 EDA 연습용으로 사용해보자.
- 퓨쳐스킬 과제로 진행

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/kaggle/input/kaggle-survey-2020/kaggle_survey_2020_responses.csv')

In [ ]:
df

In [ ]:
# df 미리보기는 답답하다.. 어떤 질문들이 있는지 확인해보자.
for col, question in zip(df.columns, df.iloc[0]):
    print(f'{col} ::: {question}')

In [ ]:
df.describe()

궁금한 내용
- 주로 사용하는 언어
- 직무별 배우고싶은 categories of automated machine learning tools
- 직무별로 ML직접 사용비율/tool 사용비율이 다를까?

## 설문자 나이/성별/직무 분포

In [ ]:
age_x = df.iloc[1:].groupby(by='Q1')['Q1'].count().index
age_y = df.iloc[1:].groupby(by='Q1')['Q1'].count().values

In [ ]:
gender_x = df.iloc[1:].groupby(by='Q2')['Q2'].count().index
gender_y = df.iloc[1:].groupby(by='Q2')['Q2'].count().values

In [ ]:
role_x = df.iloc[1:].groupby(by='Q5')['Q5'].count().index
role_y = df.iloc[1:].groupby(by='Q5')['Q5'].count().values

In [ ]:
fig, (ax_age, ax_gender) = plt.subplots(1, 2, figsize=(20, 4))

ax_age.bar(age_x, age_y)
ax_age.set_title('age count')

ax_gender.bar(gender_x, gender_y)
ax_gender.set_title('gender count')

plt.show()

In [ ]:
fig, ax_role = plt.subplots(figsize=(10, 4))

colors = sns.color_palette('hls', len(role_x)) 
bars = ax_role.bar(role_x, role_y, color=colors)
ax_role.set_title('role count')
ax_role.legend(bars, role_x, loc='upper left', bbox_to_anchor=(1, 1))
ax_role.get_xaxis().set_visible(False)

plt.show()

- xticks label이 긴 경우 그냥 legend로 빼버리는게 나은것같다.
- 학생이 압도적으로 많다..
- 학생을 제외하면 데싸-소프트웨어 엔지니어-기타등등

## 주 사용 언어

In [ ]:
df.loc[:, 'Q7_Part_1':'Q7_OTHER']

- 다중선택 질문의 경우 각 선택지마다 column이 있고, 해당 항목을 선택하지 않은 경우 None이 수집된다.
- 이 df에 count를 해주면 None이 아닌 값만 count된다. 

In [ ]:
lang_x = ['Python','R','SQL','C','C++','Java','Javascript','Julia','Swift','Bash','MATLAB','None','Other']
lang_y = df.loc[:, 'Q7_Part_1':'Q7_OTHER'].count()

fig, ax_lang = plt.subplots(figsize=(10, 4))

ax_lang.bar(lang_x, lang_y)
ax_lang.set_title('lang count')

plt.show()

- 예상대로 python이 압도적, 캐글 설문답게 SQL도 우세
- 직무별 추천하는 언어가 다를까?

In [ ]:
df.iloc[1:].groupby(['Q5', 'Q8'])['Q8'].count()

- 직무별, 언어별로 일단 count를 해본다.
- multi index로 들어가있는 언어를 column으로 돌리기 위해 unstack을 해준다. 

In [ ]:
df.iloc[1:].groupby(['Q5', 'Q8'])['Q8'].count().unstack()

- NaN값이 많고, 단순 카운트는 직무별 응답자 수 차이가 있어 값을 정규화해야 보기 좋을 것 같다.
- 각 값을 직무별 응답자 수에 대한 비율로 만들어주기 위해, 각 직무별 sum을 먼저 구한 후 그 값으로 나눠준다.

In [ ]:
lang_by_role_df = df.iloc[1:].groupby(['Q5', 'Q8'])['Q8'].count().unstack()
per_lang_by_role = lang_by_role_df.div(lang_by_role_df.sum(axis=1), axis=0)

In [ ]:
per_lang_by_role

In [ ]:
fig, ax_lang_role = plt.subplots(figsize=(10, 10))
ax_lang_role = sns.heatmap(per_lang_by_role, vmin=-0.1)
plt.title('language recommendation', fontsize=20)
plt.show() 

- 파이썬이 압도적
- 상대적으로 Business analyst, Data analyst, Data scientist, Research scientist가 다른 직무에 비해 R을 추천하는 경향

## Tool 관련

In [ ]:
tool_x = ['data augmentation','feature engineering','model selection','model architecture searches','hyperparameter tuning','full ML pipelines','None','Other']
tool_y = df.loc[:, 'Q33_B_Part_1':'Q33_B_OTHER'].count()

fig, ax_tool = plt.subplots(figsize=(5, 5))

colors = sns.color_palette('hls', len(tool_x)) 
bars = ax_tool.bar(tool_x, tool_y, color=colors)
ax_tool.set_title('tool count')
ax_tool.legend(bars, tool_x, loc='upper left', bbox_to_anchor=(1, 1))
ax_tool.get_xaxis().set_visible(False)

plt.show()

- 전반적으로 관심이 높지만 model selection, pipeline에 관심도가 높다

In [ ]:
# 각 분야의 직무별 비율
df_tool_and_role = df.loc[1:, ['Q5', 'Q33_B_Part_1', 'Q33_B_Part_2', 'Q33_B_Part_3', 'Q33_B_Part_4', 'Q33_B_Part_5', 'Q33_B_Part_6', 'Q33_B_Part_7', 'Q33_B_OTHER']].groupby('Q5').count()
df_tool_and_role = df_tool_and_role.swapaxes(0, 1)
df_tool_and_role.index = tool_x
colors = sns.color_palette('hls', len(df_tool_and_role.columns)) 
df_tool_and_role.plot.bar(stacked=True, figsize=(10, 10), color=colors)

- 직군 수에 차이가 있어 눈에 띄지 않는다. 비율로 바꿔보자
- not employed와 Other, student와 수가 적은 statistician, DBA는 빼보자

In [ ]:
per_df_tool_and_role = df_tool_and_role.div(df_tool_and_role.sum(axis=0), axis=1)
per_df_tool_and_role = per_df_tool_and_role.iloc[:, [0, 3, 4, 5, 6, 8, 9, 10]]
colors = sns.color_palette('hls', len(per_df_tool_and_role.columns))
per_df_tool_and_role.plot.bar(stacked=True, figsize=(10, 10), color=colors)

- 줄무늬 도마뱀같다.
- 대부분 고른 분포.. 별 차이는 없었다
- 뭘 배우고 싶지 않은 PM들이 많다.
- ml 엔지니어들이 다른 직무에 비해 data augmentation이나 model architecture search를 배우고 싶어하고, None으로 응답한 비율이 가장 적다. 학구열이 가장 높은 집단 아닐까? 생존을 위해 배워야 한다거나

In [ ]:
# 자연어처리 모델은 뭘 쓰는지도 궁금하다
models_x = ['Word embeddings/vectors', 'Encoder-decorder models', 'Contextualized embeddings', 'Transformer language models', 'None', 'Other']
models_y = df[1:].loc[:, 'Q19_Part_1':'Q19_OTHER'].count().values

fig, ax = plt.subplots(figsize=(5, 5))
colors = sns.color_palette('hls', len(models_x))
bars = ax.bar(models_x, models_y, color=colors)
ax.legend(bars, models_x, loc='upper left', bbox_to_anchor=(1, 1))
ax.get_xaxis().set_visible(False)
plt.show()

- 아직은 word embedding도 죽지 않았다.. 가성비도 중요한것같다